In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Essential_Thrombocythemia/GSE159514'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression profiling of granulocytes from primary and secondary myelofibrosis"
!Series_summary	"Primary myelofibrosis (PMF) together with polycythemia vera (PV) and essential thrombocythemia (ET) belongs to the classic Philadelphia-negative myeloproliferative neoplasms (MPNs). PV and ET can evolve to secondary myelofibrosis (SMF) giving rise to post-PV (PPV) and post-ET (PET) myelofibrosis (MF). PMF and SMF patients are currently managed in the same way and prediction of survival is based on the same prognostic models, even if it has been demonstrated that they can’t accurately distinguish different risk categories in SMF. In the last few years interest grew concerning the ability of gene expression profiling (GEP) to provide valuable prognostic information for clinical decision making. To construct a molecular signature that can predict survival according to gene expression we studied GEP of granulocytes from 114 MF patients, including 35 pr

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Determine the availability of gene expression data
# The dataset summary suggests that data was obtained using an Affymetrix platform, indicating gene expression data
is_gene_available = True

# Identify the trait, age, and gender rows in the sample characteristics dictionary
sample_characteristics = {
    0: ['disease: PPV', 'disease: Overt-PMF', 'disease: PET', 'disease: Pre-PMF'],
    1: ['driver mutation: JAK2V617F', 'driver mutation: CALR Type 1', 'driver mutation: MPL', 'driver mutation: TN', 'driver mutation: CALR Type 2', 'driver mutation: CALR', 'driver mutation: JAK2 ex12']
}

# Key 0 refers to disease type which is needed to identify Essential Thrombocythemia (ET)
if 'disease: PET' in sample_characteristics[0]:
    trait_row = 0

# `age` and `gender` information is not explicitly mentioned in the provided sample characteristics dictionary
# Thus, we set age_row and gender_row to None since data is not available for these variables
age_row = None
gender_row = None

# Define the conversion functions
def convert_trait(value):
    trait_mapping = {
        'PPV': None,
        'Overt-PMF': None,
        'PET': 1,  # Essential Thrombocythemia
        'Pre-PMF': None
    }
    return trait_mapping.get(value.split(': ')[1], None)

# Save cohort information
save_cohort_info('GSE159514', './preprocessed/Essential_Thrombocythemia/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Essential_Thrombocythemia', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Essential_Thrombocythemia/trait_data/GSE159514.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM4831515': [None], 'GSM4831516': [None], 'GSM4831517': [1], 'GSM4831518': [None], 'GSM4831519': [None], 'GSM4831520': [None], 'GSM4831521': [1], 'GSM4831522': [None], 'GSM4831523': [None], 'GSM4831524': [None], 'GSM4831525': [None], 'GSM4831526': [None], 'GSM4831527': [None], 'GSM4831528': [None], 'GSM4831529': [None], 'GSM4831530': [None], 'GSM4831531': [None], 'GSM4831532': [None], 'GSM4831533': [1], 'GSM4831534': [None], 'GSM4831535': [None], 'GSM4831536': [1], 'GSM4831537': [None], 'GSM4831538': [None], 'GSM4831539': [None], 'GSM4831540': [None], 'GSM4831541': [1], 'GSM4831542': [None], 'GSM4831543': [1], 'GSM4831544': [None], 'GSM4831545': [None], 'GSM4831546': [1], 'GSM4831547': [1], 'GSM4831548': [None], 'GSM4831549': [None], 'GSM4831550': [1], 'GSM4831551': [1], 'GSM4831552': [1], 'GSM4831553': [None], 'GSM4831554': [None], 'GSM4831555': [1], 'GSM4831556': [None], 'GSM4831557': [None], 'GSM4831558': [None], 'GSM4831559': [None], 'GSM4831560': [1], 'GSM4831561': [None], 'GSM

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['11715100_at', '11715101_s_at', '11715102_x_at', '11715103_x_at',
       '11715104_s_at', '11715105_at', '11715106_x_at', '11715107_s_at',
       '11715108_x_at', '11715109_at', '11715110_at', '11715111_s_at',
       '11715112_at', '11715113_x_at', '11715114_x_at', '11715115_s_at',
       '11715116_s_at', '11715117_x_at', '11715118_s_at', '11715119_s_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['11715100_at', '11715101_s_at', '11715102_x_at', '11715103_x_at', '11715104_s_at'], 'GeneChip Array': ['Human Genome HG-U219 Array', 'Human Genome HG-U219 Array', 'Human Genome HG-U219 Array', 'Human Genome HG-U219 Array', 'Human Genome HG-U219 Array'], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['20-Aug-10', '20-Aug-10', '20-Aug-10', '20-Aug-10', '20-Aug-10'], 'Sequence Type': ['Consensus sequence', 'Consensus sequence', 'Consensus sequence', 'Consensus sequence', 'Consensus sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database'], 'Transcript ID(Array Design)': ['g21264570', 'g21264570', 'g21264570', 'g22748780', 'g30039713'], 'Target Description': ['g21264570 /TID=g21264570 /CNT=1 /FEA=FLmRNA /TIER=FL /STK=0 /DEF=g21264

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Decide the keys for the identifiers and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'Gene Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to get the gene expression dataframe
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(preview_df(gene_data))


{'GSM4831515': [4.01576, 2.37927, 2.47963, 2.27795, 2.60671], 'GSM4831516': [3.92917, 2.85102, 2.48319, 2.3183575, 2.53692], 'GSM4831517': [4.19574, 3.17362, 2.4773, 2.3587525, 2.63595], 'GSM4831518': [3.76736, 2.51469, 2.67008, 2.1824525, 2.37125], 'GSM4831519': [3.92954, 2.35293, 2.55532, 2.3112475, 2.50474], 'GSM4831520': [4.03084, 2.30575, 2.65001, 2.310885, 2.62355], 'GSM4831521': [4.0512, 3.10341, 2.58174, 2.3344925, 2.51353], 'GSM4831522': [4.20872, 2.29862, 2.63154, 2.159875, 2.35302], 'GSM4831523': [4.05559, 2.79829, 2.77868, 2.31867, 2.65773], 'GSM4831524': [4.05048, 2.30246, 2.67626, 2.3152925, 2.49283], 'GSM4831525': [3.68885, 2.82863, 3.04848, 2.1783775, 2.32002], 'GSM4831526': [3.90686, 2.67509, 2.63847, 2.2613625, 2.48892], 'GSM4831527': [4.15014, 3.43957, 2.54389, 2.2861625, 2.69051], 'GSM4831528': [3.9806, 2.15555, 2.77982, 2.3338799999999997, 2.64329], 'GSM4831529': [4.15207, 2.53903, 3.0267, 2.301345, 2.91752], 'GSM4831530': [4.2189, 2.88498, 2.69024, 2.3486925, 2.69

### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Essential_Thrombocythemia/gene_data/GSE159514.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Essential_Thrombocythemia')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE159514', './preprocessed/Essential_Thrombocythemia/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Essential_Thrombocythemia/GSE159514.csv'
    unbiased_merged_data.to_csv(csv_path)


Quartiles for 'Essential_Thrombocythemia':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1.0
Max: 1.0
The distribution of the feature 'Essential_Thrombocythemia' in this dataset is severely biased.

